<a href="https://colab.research.google.com/github/Takutino/AIT_finalProject_milestone/blob/main/AIT_project_milestone_takuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Maestro midi files

In [1]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip

--2021-03-25 05:48:54--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 173.194.215.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: ‘maestro-v3.0.0-midi.zip’

maestro-v3.0.0-midi 100%[===================>]  55.71M   102MB/s    in 0.5s    

2021-03-25 05:48:55 (102 MB/s) - ‘maestro-v3.0.0-midi.zip’ saved [58416533/58416533]



Unzip maestro midi files

In [2]:
!apt-get install unp
!unp maestro-v3.0.0-midi.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  xdms cabextract orange
The following NEW packages will be installed:
  unp
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 17.2 kB of archives.
After this operation, 136 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 unp all 2.0~pre7+nmu1 [17.2 kB]
Fetched 17.2 kB in 0s (64.3 kB/s)
Selecting previously unselected package unp.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../unp_2.0~pre7+nmu1_all.deb ...
Unpacking unp (2.0~pre7+nmu1) ...
Setting up unp (2.0~pre7+nmu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Archive:  maestro-v3.0.0-midi.zip
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG_MID--AUDIO_08_R1_2004_01_Track01_wav.midi  
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_09_R1_2004_05_

Install mido for working with MIDI messages and ports

In [3]:
!pip install mido

     |████████████████████████████████| 61kB 2.7MB/s 


Download the file which has all the meta information of midi files

In [4]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.csv

--2021-03-25 05:49:09--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 74.125.31.128, 74.125.26.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298977 (292K) [application/octet-stream]
Saving to: ‘maestro-v3.0.0.csv’

maestro-v3.0.0.csv  100%[===================>] 291.97K  --.-KB/s    in 0.003s  

2021-03-25 05:49:09 (95.0 MB/s) - ‘maestro-v3.0.0.csv’ saved [298977/298977]



Read the csv file in the form of two dimensional datas, which will be used for training, validation and testing

In [5]:
# source: https://github.com/DRyanMiller/MIDI_Visualization
import pandas as pd
# python package : working with panel datas : 2 dimensional data
files = pd.read_csv('https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.csv')
# will be using this for training and validation, and testing

Install Unidecode for encoding

In [6]:
# this is the one we are most likely to use
!pip install Unidecode

     |████████████████████████████████| 245kB 4.4MB/s 


Read midi files and define some methods such as 
getting the max and min pitch of midi file and getting the time dimension for a pianoroll given a certain quantization


In [7]:
from mido import MidiFile
from unidecode import unidecode
import numpy as np

#######
# Pianorolls dims are  :   TIME  *  PITCH

#quantization : amplitude  resolution 
class Read_midi(object):
# initialize the some of properties
    def __init__(self, song_path, quantization):
        ## Metadata
        self.__song_path = song_path
        self.__quantization = quantization

        ## Pianoroll
        self.__T_pr = None

        ## Private misc
        self.__num_ticks = None
        self.__T_file = None

    @property
    def quantization(self):
        return self.__quantization

    @property
    def T_pr(self):
        return self.__T_pr

    @property
    def T_file(self):
        return self.__T_file

    def get_total_num_tick(self):
        # Midi length should be written in a meta message at the beginning of the file,
        # Read a midi file and return a dictionnary {track_name : pianoroll}
        mid = MidiFile(self.__song_path)

        # Parse track by track
        num_ticks = 0
        for i, track in enumerate(mid.tracks):
            tick_counter = 0
            for message in track:
                # Note on
                time = float(message.time)
                tick_counter += time
            num_ticks = max(num_ticks, tick_counter)
        self.__num_ticks = num_ticks

    # it finds the min and max pitch
    def get_pitch_range(self):
        mid = MidiFile(self.__song_path)
        min_pitch = 200
        max_pitch = 0
        for i, track in enumerate(mid.tracks):
            for message in track:
                if message.type in ['note_on', 'note_off']:
                    pitch = message.note
                    if pitch > max_pitch:
                        max_pitch = pitch
                    if pitch < min_pitch:
                        min_pitch = pitch
        return min_pitch, max_pitch
    
    def get_time_file(self):
        # Get the time dimension for a pianoroll given a certain quantization
        mid = MidiFile(self.__song_path)
        # Tick per beat
        ticks_per_beat = mid.ticks_per_beat
        # Total number of ticks
        self.get_total_num_tick()
        # Dimensions of the pianoroll for each track -> equation for getting the time
        self.__T_file = int((self.__num_ticks / ticks_per_beat) * self.__quantization)
        return self.__T_file

    # Read the actual data and read the score and tone
    def read_file(self):
        # Read the midi file and return a dictionnary {track_name : pianoroll}
        mid = MidiFile(self.__song_path)
        # Tick per beat
        ticks_per_beat = mid.ticks_per_beat

        # Get total time
        self.get_time_file()
        T_pr = self.__T_file
        #print("T_pr:", T_pr)
        # Pitch dimension
        N_pr = 128
        pianoroll = {}

        def add_note_to_pr(note_off, notes_on, pr):
            pitch_off, _, time_off = note_off
            # Note off : search for the note in the list of note on,
            # get the start and end time
            # write it in th pr
            match_list = [(ind, item) for (ind, item) in enumerate(notes_on) if item[0] == pitch_off]
            if len(match_list) == 0:
                print("Try to note off a note that has never been turned on")
                # Do nothing
                return

            # Add note to the pr
            pitch, velocity, time_on = match_list[0][1]
            pr[time_on:time_off, pitch] = velocity
            #v :  the force how strong you push the key
            #pr contains np array that has music data
            # Remove the note from notes_on
            ind_match = match_list[0][0]
            del notes_on[ind_match]
            return

        # Parse track by track
        counter_unnamed_track = 0
        #print("Number of tracks: ",mid.tracks)
        for i, track in enumerate(mid.tracks):
            # Instanciate the pianoroll
            pr = np.zeros([T_pr, N_pr])
            time_counter = 0
            notes_on = []
            for message in track:

                ##########################################
                ##########################################
                ##########################################
                # TODO : keep track of tempo information
                # import re
                # if re.search("tempo", message.type):
                #     import pdb; pdb.set_trace()
                ##########################################
                ##########################################
                ##########################################


                # print message
                # Time. Must be incremented, whether it is a note on/off or not
                time = float(message.time)
                time_counter += time / ticks_per_beat * self.__quantization
                # Time in pr (mapping)
                time_pr = int(round(time_counter))
                # Note on
                if message.type == 'note_on':
                    # Get pitch
                    pitch = message.note
                    # Get velocity
                    velocity = message.velocity
                    if velocity > 0:
                        notes_on.append((pitch, velocity, time_pr))
                    elif velocity == 0:
                        add_note_to_pr((pitch, velocity, time_pr), notes_on, pr)
                # Note off
                elif message.type == 'note_off':
                    pitch = message.note
                    velocity = message.velocity
                    add_note_to_pr((pitch, velocity, time_pr), notes_on, pr)

            # We deal with discrete values ranged between 0 and 127
            #     -> convert to int
            pr = pr.astype(np.int16)
            if np.sum(np.sum(pr)) > 0:
                name = unidecode(track.name)
                name = name.rstrip('\x00')
                if name == u'':
                    name = 'unnamed' + str(counter_unnamed_track)
                    counter_unnamed_track += 1
                if name in pianoroll.keys():
                    # Take max of the to pianorolls
                    pianoroll[name] = np.maximum(pr, pianoroll[name])
                else:
                    pianoroll[name] = pr
        return pianoroll

        #goal : return the np array : 2d matrices : time * pitch , assign values [velocity(loudness)]

import numpy as np

def get_pianoroll_time(pianoroll):
    T_pr_list = []
    for k, v in pianoroll.items():
        T_pr_list.append(v.shape[0])
    if not len(set(T_pr_list)) == 1:
        print("Inconsistent dimensions in the new PR")
        return None
    return T_pr_list[0]

def get_pitch_dim(pianoroll):
    N_pr_list = []
    for k, v in pianoroll.items():
        N_pr_list.append(v.shape[1])
    if not len(set(N_pr_list)) == 1:
        print("Inconsistent dimensions in the new PR")
        raise NameError("Pr dimension")
    return N_pr_list[0]

def dict_to_matrix(pianoroll):
    T_pr = get_pianoroll_time(pianoroll)
    N_pr = get_pitch_dim(pianoroll)
    rp = np.zeros((T_pr, N_pr), dtype=np.int16)
    for k, v in pianoroll.items():
        rp = np.maximum(rp, v)
    return rp

Go through all the midi files, load them onto numpy arrays split the data based on splic column of the datas

In [9]:
# go through all the midi files based on csv files
midi_files = []

midi_files_train = []
midi_files_valid = []
midi_files_test = []

for i in range(len(files)):
  # files: that has the content of csv file
  # files[""] select the column of that data flame
  #.iloc selects the specific row
  file_path = "maestro-v3.0.0/"
  file_name = files["midi_filename"].iloc[i]
  fileForOneRow = files.iloc[i]
  
  m = Read_midi(file_path+file_name, 4).read_file()
  matrixBeforeSplit = dict_to_matrix(m)

  print(file_name)

  if (files["split"].iloc[i] == "train") : 
      midi_files_train.append(matrixBeforeSplit)
  elif (files["split"].iloc[i] == "validation") :
      midi_files_valid.append(matrixBeforeSplit)
  else :
      midi_files_test.append(matrixBeforeSplit)

print("For training set")
print(len(midi_files_train))
print("For test set")
print(len(midi_files_test))
print("For validation set")
print(len(midi_files_valid))

2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi
2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi
2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi
2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi
2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi
2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi
2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi
2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi
2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi
2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.midi
2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_09_Track09_wav.midi
2006/MIDI-Unprocessed_19_R1_2006_01-07_ORIG_MID--AUDIO_19_R1_2006_04_Track04_wav.midi
2013/ORIG-MIDI_02_7_1